In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os,cv2
import torch.nn as nn
from cyvlfeat.kmeans import kmeans,kmeans_quantize
from losses import relu_evidence

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
feats=np.load('record_feats_test.npy').astype(np.float32)
data=np.reshape(feats,(feats.shape[0],-1))
uncertianty=True

In [2]:
feats=np.load('record_feats_train.npy').astype(np.float32)
data=np.reshape(feats,(feats.shape[0],-1))

if  not os.path.exists('sparse_labels.npy'):
    print('clustering')
    K=10
    centers=kmeans(data,num_centers=K,initialization='PLUSPLUS',num_repetitions=10,
                   max_num_comparisons=100,max_num_iterations=100,algorithm='LLOYD',num_trees=3)
    labels=kmeans_quantize(data,centers)
    # to get the sparse matrix of labels
    sparse_labels=np.eye(K)[labels]
    labels=labels.astype(np.float32)
    sparse_labels=sparse_labels.astype(np.float32)

    np.save('sparse_labels.npy',sparse_labels)
    np.save('labels.npy',labels)
else:
    sparse_labels=np.load('sparse_labels.npy').astype(np.float32)
    labels=np.load('labels.npy').astype(np.float32)


In [3]:
BATCH_SIZE=64
train_data_loadpath='/mnt/sda/sunche/datasets/Anomaly/Avenue/test/'
train_label_loadpath='/mnt/sda/sunche/datasets/Anomaly/Avenue/avenue_gts.txt'
def Gaussian(X,rate=0.05):
    # Salt and pepper noise
    drop = np.random.normal(0,rate, X.shape)
    X=X+drop
    return X
class MyDataset(Dataset):
    def __init__(self, train_data_loadpath,labels,transform=None, target_transform=None,):
        self.path=train_data_loadpath
        self.files=os.listdir(train_data_loadpath)
        self.inds=range(len(self.files))
        self.gts=np.loadtxt(train_label_loadpath).astype(np.float32)
        self.labels=labels
    def __getitem__(self, index):
        img_path=os.path.join(self.path,str(self.inds[index]+1)+'.jpg')
        img = cv2.resize(cv2.imread(img_path),(224,224))
        img_npy=np.array(img).astype(np.float32)/255.0
        
#         img_npy = Gaussian(img_npy)
        img_npy = np.reshape(img_npy,[224,224,3])
        img_npy = np.transpose(img_npy,[2,0,1])
        label=self.labels[index]

        img_npy=torch.from_numpy(img_npy).float()
#         label=torch.from_numpy(label).float()
        return img_npy,label

        return img_npy,label

    def __len__(self):
        return len(self.files)
    
# torch.set_num_threads(1)
# train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE,num_workers=16, shuffle=True,pin_memory=False)  


# class MyDataset(Dataset):
#     def __init__(self, feats, labels):
#         self.feats=feats
#         self.labels=labels
#     def __getitem__(self, index):
#         return self.feats[index],self.labels[index]
#     def __len__(self):
#         return len(self.feats)
BATCH_SIZE=32
if uncertianty==False:
    train_data=MyDataset(train_data_loadpath,labels)
if uncertianty==True:
    train_data=MyDataset(train_data_loadpath,labels)

torch.set_num_threads(1)
train_loader = DataLoader(dataset=train_data, batch_size=1,num_workers=1)  

In [4]:
num_inputs, num_outputs, num_hiddens = data.shape[1], 10, 256
import auto_encoders
model = auto_encoders.VGG_11(20)

epoch_=9

model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.cuda()
if uncertianty==False:
    state = torch.load('./savepath_lstm_Net_6/cls_model'+str(epoch_+1)+'.pkl')['model_state']
if uncertianty==True:
    state = torch.load('./savepath_lstm_Net_6/cls_evi_model'+str(epoch_+1)+'.pkl')['model_state']
model.load_state_dict(state)
model.eval()

DataParallel(
  (module): VGG(
    (conv3_64): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv3_128): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv3_256): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (conv3_512a): Sequential(
      (0): Conv2d(256, 512, kernel_s

In [5]:
write_loss=np.zeros((0,1))
for epoch in range(1):
    print('epoch {}'.format(epoch + 1))
    # training-----------------------------
    train_loss = 0.
    for i,(batch_x, batch_y) in enumerate(train_loader):
      batch_x = batch_x.cuda()
      batch_y = batch_y.cuda()
      out= model(batch_x)
      _, preds = torch.max(out, 1)
      if uncertianty==False:
          scores=-np.max(out.data.cpu().numpy(),1)
      if uncertianty==True:
          evidence = relu_evidence(out)
          alpha = evidence + 1
          u = 20 / torch.sum(alpha, dim=1, keepdim=True)
#           print(u.data.cpu().numpy().shape)
          scores=np.max(u.data.cpu().numpy(),1)-np.mean(out.data.cpu().numpy(),1)
      record_scores=scores
#       for j in range(16):
#           record_scores.append(scores)
      write_loss=np.concatenate((write_loss,np.reshape(np.array(record_scores),(1,1))),0)
if uncertianty==False:
    np.save("total_loss_cls.npy",np.array(write_loss))
else:
    np.save("total_loss_evi_t.npy",np.array(write_loss))
# exit()
print('finished')

epoch 1
finished
